## Outbreak of Coronavirus(COVID-19): <br>
### Overview <br>
Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.

Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.  Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness.

The best way to prevent and slow down transmission is be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face. 

The COVID-19 virus spreads primarily through droplets of saliva or discharge from the nose when an infected person coughs or sneezes, so it’s important that you also practice respiratory etiquette (for example, by coughing into a flexed elbow).

At this time, there are no specific vaccines or treatments for COVID-19. However, there are many ongoing clinical trials evaluating potential treatments. WHO will continue to provide updated information as soon as clinical findings become available.. <br>


#### Common symptoms include:
* fever
* tiredness
* dry cough. 
* shortness of breath
* aches and pains
* sore throat
* very few people will report diarrhoea, nausea or a runny nose. <br>

###  Prevention <br>
To prevent infection and to slow transmission of COVID-19, do the following:

* Wash your hands regularly with soap and water, or clean them with alcohol-based hand rub.
* Maintain at least 1 metre distance between you and people coughing or sneezing.
* Avoid touching your face.
* Cover your mouth and nose when coughing or sneezing.
* Stay home if you feel unwell.
* Refrain from smoking and other activities that weaken the lungs.
* Practice physical distancing by avoiding unnecessary travel and staying away from large groups of people.



<h1 style="text-align:center;font-size:150%;;"></h1>
<img src="https://www.who.int/images/default-source/searo---images/emergencies/covid19/how-to-protect/covid-how-to-protect-yourself-02.gif?sfvrsn=c78ba713_2/" width="350" height="350" align="center"/> 


# 1.Exploratory Data Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
pd.set_option("display.max_columns",None) 
pd.set_option("display.max_rows",None) 
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
sns.set(style="darkgrid", palette="pastel", color_codes=True)
sns.set_context("paper")
from pandas_profiling import ProfileReport


import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import plotly.figure_factory as ff



py.init_notebook_mode(connected=True)
import plotly.io as pio
pio.templates.default = "seaborn"
from plotly.subplots import make_subplots


import os

In [ ]:
covid_data = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv',parse_dates=['Date'])
covid_data['Date'] = pd.DatetimeIndex(covid_data['Date']).date
covid_data['Active'] = covid_data['Confirmed']-covid_data['Recovered']-covid_data['Deaths']
covid_measures = pd.read_csv('/kaggle/input/covid19-containment-and-mitigation-measures/COVID 19 Containment measures data.csv',parse_dates = ['Date Start'])

In [ ]:
covid_data.head()

In [ ]:
covid_measures.head()

In [ ]:
profile = ProfileReport(covid_data, title='COVID Profiling Report', html={'style':{'full_width':True}})


In [ ]:
profile_measure = ProfileReport(covid_measures, minimal=True)

In [ ]:
profile.to_widgets()

In [ ]:
profile_measure.to_widgets()

In [ ]:
covid_data.rename(columns={'Date': 'date', 
                     'Province/State':'state',
                     'Country/Region':'country',
                     'Lat':'lat', 'Long':'long',
                     'Confirmed': 'confirmed',
                     'Deaths':'deaths',
                     'Recovered':'recovered'
                    }, inplace=True)

In [ ]:
df = covid_data.sort_values(by = ['date'],ascending=False)

In [ ]:
df1 = covid_data
df1['date'] = pd.to_datetime(df1['date'])
df1['date'] = df1['date'].dt.strftime('%m/%d/%Y')
df1 = df1.fillna('-')
fig = px.density_mapbox(df1, lat='lat', lon='long', z='confirmed', radius=100,zoom=1, hover_data=["country",'state',"confirmed"],
                        mapbox_style="carto-positron", animation_frame = 'date', range_color= [0, 1000],title='Spread of Covid-19')
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

In [ ]:
df.plot('date',['confirmed', 'deaths', 'recovered', 'Active'],figsize=(10,10), rot=30)

In [ ]:
plt.figure(figsize= (15,10))
plt.xticks(rotation = 90 ,fontsize = 10)
plt.yticks(fontsize = 15)
plt.xlabel("Dates",fontsize = 30)
plt.ylabel('Total cases',fontsize = 30)
plt.title("Worldwide Confirmed Cases Over Time" , fontsize = 30)
total_cases = covid_data.groupby('date')['date', 'confirmed'].sum().reset_index()
total_cases['date'] = pd.to_datetime(total_cases['date'])


ax = sns.lineplot( x = total_cases.date.dt.date ,y = total_cases.confirmed , color = 'b')
ax.set(xlabel='Dates', ylabel='Total cases')

In [ ]:
df2 = pd.pivot_table(covid_data, values=['confirmed','deaths','Active'], index='country', aggfunc='max')
df2['Death Rate'] = round(df2['deaths'] /df2['confirmed'], 2)
df2 = df2.sort_values(by='confirmed', ascending= False)
df2.style.background_gradient(cmap='Blues')

In [ ]:
top_actives = covid_data.groupby(by = 'country')['Active'].sum().sort_values(ascending = False).head(10).reset_index()
fig = px.bar(top_actives, 
             x="Active",
             y="country", 
             orientation='h',
             height=800,
             title='Active Cases insight')
fig.show()

In [ ]:
fig = px.line(df, x="date", y="confirmed", color='country', height=600,
             title='Confirmed', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()


fig = px.line(df, x="date", y="deaths", color='country', height=600,
             title='deaths', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()


fig = px.line(df, x="date", y="Active", color='country', height=600,
             title='Active', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

In [ ]:
covid_data_sun = covid_data.tail(20)
fig = px.sunburst(covid_data_sun, path=["country"],
                  color='country', hover_data=['country'],
                  color_continuous_scale='rainbow')
fig.show()

In [ ]:
grp = covid_data.groupby(['country'])['confirmed', 'deaths', 'recovered'].max()
grp = grp.reset_index()
grp['Country'] =  grp['country']
fig = px.choropleth(grp, locations="country", locationmode='country names', 
                     color="confirmed", hover_name="country",hover_data = [grp.recovered,grp.deaths],projection="orthographic",
                     color_continuous_scale='Reds')
fig.update(layout_coloraxis_showscale=True)

In [ ]:
grp = df.groupby(['date', 'country'])['confirmed', 'deaths', 'recovered','Active'].max()
grp = grp.reset_index()
grp['Date'] = pd.to_datetime(grp['date'])
grp['Date'] = grp['Date'].dt.strftime('%m/%d/%Y')

fig = px.choropleth(grp, locations="country", locationmode='country names', 
                     color="confirmed", hover_name="country",hover_data = [grp.recovered,grp.deaths,grp.Active],projection="natural earth",
                     animation_frame="Date",width=1000, height=700,
                     color_continuous_scale='blues',
                     range_color=[1000,50000],

                     title='World Map of COVID-19 Spread')

fig.update(layout_coloraxis_showscale=True)
py.offline.iplot(fig)

In [ ]:
fig = px.treemap(df, path=['country'], values='recovered',
                  color='recovered', hover_data=['country'],
                  color_continuous_scale='Rainbow')
fig.show()
fig = px.treemap(df, path=['country'], values='deaths',
                  color='deaths', hover_data=['country'],
                  color_continuous_scale='rdbu')
fig.show()
fig = px.treemap(df, path=['country'], values='Active',
                  color='Active', hover_data=['country'],
                  color_continuous_scale='earth')
fig.show()

In [ ]:
covid_measures.head()

In [ ]:
covid_measures.columns = covid_measures.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


In [ ]:
fig,ax=plt.subplots(figsize=(15,5))
sns.heatmap(covid_measures.isnull(), annot=True)

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator
covid_measures['description_of_measure_implemented'] = covid_measures['description_of_measure_implemented'].fillna('')
text = " ".join(str(each) for each in covid_measures.description_of_measure_implemented)
wordcloud = WordCloud(max_words=200,colormap='Set1', background_color="black").generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

In [ ]:
grp_count = df.groupby(["country"])[["Active"]].mean().reset_index()

fig = px.pie(grp_count,
             values="country",
             names="Active",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

In [ ]:
sns.pairplot(df);

### Strategical Analysis for COVID-19

In [ ]:
china_strat = covid_measures[covid_measures['country'] == 'China']['keywords'].value_counts().head(10)
china_strat
fig = px.bar(china_strat, x = china_strat.index, y = china_strat.values)
fig.update_layout(title = 'Top Strategies Used by China')
fig.show()

In [ ]:
it_strat = covid_measures[covid_measures['country'] == 'Italy']['keywords'].value_counts().head(10)
it_strat
fig = px.bar(it_strat, x = it_strat.index, y = it_strat.values)
fig.update_layout(title = 'Top Strategies Used by Italy')
fig.show()

In [ ]:
India_strat = covid_measures[covid_measures['country'] == 'India']['keywords'].value_counts().head(10)
India_strat
fig = px.bar(India_strat, x = India_strat.index, y = India_strat.values)
fig.update_layout(title = 'Top Strategies Used by India')
fig.show()

In [ ]:
us_strat = covid_measures[covid_measures['country'] == 'US:New York']['keywords'].value_counts().head(10)
us_strat
fig = px.bar(us_strat, x = us_strat.index, y = us_strat.values)
fig.update_layout(title = 'Top Strategies Used by US(NY)')
fig.show()

In [ ]:
us_strat = covid_measures[covid_measures['country'] == 'United States']['keywords'].value_counts().head(10)
us_strat
fig = px.bar(us_strat, x = us_strat.index, y = us_strat.values)
fig.update_layout(title = 'Top Strategies Used by US')
fig.show()

In [ ]:
uk_strat = covid_measures[covid_measures['country'] == 'United Kingdom']['keywords'].value_counts().head(10)
uk_strat
fig = px.bar(uk_strat, x = uk_strat.index, y = uk_strat.values)
fig.update_layout(title = 'Top Strategies Used by UK')
fig.show()

In [ ]:

sk_strat = covid_measures[covid_measures['country'] == 'South Korea']['keywords'].value_counts().head(10)
sk_strat
fig = px.bar(sk_strat, x = sk_strat.index, y = sk_strat.values)
fig.update_layout(title = 'Top Strategies Used by South Korea')
fig.show()

In [ ]:

singa_strat = covid_measures[covid_measures['country'] == 'Singapore']['keywords'].value_counts().head(10)
singa_strat
fig = px.bar(singa_strat, x = singa_strat.index, y = singa_strat.values)
fig.update_layout(title = 'Top Strategies Used by Singapore')
fig.show()

### COVID-19 Test Analysis

In [ ]:
data_test = pd.read_csv('/kaggle/input/covid19testing/tested_worldwide.csv', delimiter=',') 
data_test.info()

In [ ]:
data_test.head()

In [ ]:
data_test=data_test.dropna(subset=['daily_tested','total_tested'])
data_test['daily_rate']=data_test.daily_positive/data_test.daily_tested*100
data_test.daily_rate.replace(np.inf, 0, inplace=True)
data_test.daily_rate.fillna(0, inplace=True)
data_test.reset_index(drop=True, inplace=True)
data_test.head()

In [ ]:
fig = px.pie(data_test,
            values="total_tested",
             names="Country_Region")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

In [ ]:
df_break = data_test.groupby(['Country_Region','Date'])['daily_tested','total_tested'].sum().reset_index()
top10case = data_test.sort_values('daily_tested', ascending=False).head(10)['Country_Region'].to_list()
top10death = data_test.sort_values('total_tested', ascending=False).head(10)['Country_Region'].to_list()

_ = df_break[df_break.Country_Region.isin(top10case)]
plt.figure(figsize=(10,6))
sns.lineplot(x='Date',y='daily_tested', data=_, hue='Country_Region', lw=2)
plt.legend(bbox_to_anchor=(1.02, 1), fontsize=10)
plt.title('Daily Test for US')
plt.show()

In [ ]:
test_10 = data_test.groupby('Country_Region')['Date'].min().sort_values().head(10).index
df= data_test[data_test.Country_Region.isin(test_10)]
plt.figure(figsize=(10,6))
sns.lineplot(x='Date',y='total_tested', data=df, hue='Country_Region', lw=2)
plt.legend(bbox_to_anchor=(1.02, 1), fontsize=10)
plt.title('Total Test conducted Country wise')
plt.show()

In [ ]:
fig = px.line(df_break,x='Date',y='daily_tested',color='Country_Region',title='Cumulative daily tested over time')
fig.update_layout(template="simple_white")
fig.show()


fig = px.line(df_break,x='Date',y='total_tested',color='Country_Region',title='Cumulative total tested over time')
fig.update_layout(template="simple_white")
fig.show()


<b> More Data Analysis in Pipeline. And don't Hesitate to give an upvote if you like it!!!! </b>